In [2]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import glob
import sys
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.manifold import TSNE
from sklearn import preprocessing
import pandas as pd
from multiprocessing import Process# this is used for multithreading
import multiprocessing
import codecs# this is used for file operations 
import random as r
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
train_benign_df = pd.read_csv("./csv_files/train_benign_asmfile.txt")
train_benign_df.drop(train_benign_df.columns[-1], axis=1, inplace=True)
train_benign_df.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
train_benign_df['label'] = 0

train_malware_df = pd.read_csv("./csv_files/train_malware_asmfile.txt")
train_malware_df.drop(train_malware_df.columns[-1], axis=1, inplace=True)
train_malware_df.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
train_malware_df['label'] = 1

test_benign_df = pd.read_csv("./csv_files/test_benign_asmfile.txt")
test_benign_df.drop(test_benign_df.columns[-1], axis=1, inplace=True)
test_benign_df.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
test_benign_df['label'] = 0

test_malware_df = pd.read_csv("./csv_files/test_malware_asmfile.txt")
test_malware_df.drop(test_malware_df.columns[-1], axis=1, inplace=True)
test_malware_df.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
test_malware_df['label'] = 1

valid_benign = pd.read_csv("./csv_files/valid_benign_asmfile.txt")
valid_benign.drop(valid_benign.columns[-1], axis=1, inplace=True)
valid_benign.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
valid_benign['label'] = 0

valid_malware = pd.read_csv("./csv_files/valid_malware_asmfile.txt")
valid_malware.drop(valid_malware.columns[-1], axis=1, inplace=True)
valid_malware.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
valid_malware['label'] = 1


In [20]:
import sklearn
df_list = [train_benign_df, train_malware_df, test_benign_df, test_malware_df, valid_benign, valid_malware]
full_df = pd.concat(df_list, ignore_index=True, axis=0)
full_shuffled_df = sklearn.utils.shuffle(full_df)
full_shuffled_df

,ID,jmp,mov,retf,push,pop,xor,retn,nop,sub,...,ebx,ecx,edi,ebp,esp,eip,.dll,std::,:dword,label
2273,05ee9db3ad5ba6e28136c798c1c84a1e,146,1378,0,1735,104,45,59,0,77,...,0,0,0,0,0,0,3,0,0,1
2774,4e30f950745a0224ac3fe6d36d97a3b2,39,708,0,749,120,116,65,3,49,...,0,0,0,0,0,0,0,0,31,1
1262,3eb4ed55a6986a5b0fb0f7604795c004,146,1378,0,1735,104,45,59,0,77,...,0,0,0,0,0,0,3,0,0,1
1087,1b92b2c710cc29b8c61868331363880d,95,708,0,790,208,63,59,2,23,...,0,0,0,0,0,0,0,0,0,1
707,0e9bef68eefd180b66bebb17261bd0ee,444,8697,0,4480,2146,709,1017,0,221,...,0,0,0,0,0,0,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,8d4df82735e0503f4000776066a5b730,1624,10289,0,5461,1473,806,599,0,595,...,0,0,0,36,0,0,7,143,56,0
2234,5ac6d0dcc66bdb0a6857cf1036c782a3,36,719,0,690,139,112,65,0,43,...,0,0,0,0,0,0,0,0,0,1
265,7d037e85cbb043a2982d300973a451f0,628,13076,0,16924,3154,1014,1096,0,169,...,0,0,0,0,0,0,0,0,0,0
492,8b1c80f4d152e1e0e6cc911193e04d1d,264,2868,0,1418,324,105,188,0,102,...,0,0,0,0,0,0,2,22,5,0


In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
NORMALIZE_VALID_COLUMNS = [ 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','lea','movzx','edi','ebp','esp', '.dll', 'std::', ':dword']

full_shuffled_df[NORMALIZE_VALID_COLUMNS] = scaler.fit_transform(full_shuffled_df[NORMALIZE_VALID_COLUMNS])
full_shuffled_df[NORMALIZE_VALID_COLUMNS].astype('float')
df = full_shuffled_df[NORMALIZE_VALID_COLUMNS]
df

,jmp,mov,retf,push,pop,xor,retn,nop,sub,inc,...,jnb,jz,lea,movzx,edi,ebp,esp,.dll,std::,:dword
2273,0.001400,0.000846,0.000000,0.002868,0.000454,0.000288,0.000689,0.000000,0.000692,0.000000,...,0.000166,0.000124,0.004739,0.000084,0.0,0.00,0.0,0.008876,0.000000,0.000000
2774,0.000374,0.000430,0.000000,0.001238,0.000523,0.000743,0.000759,0.000129,0.000440,0.000885,...,0.000166,0.000722,0.000862,0.000063,0.0,0.00,0.0,0.000000,0.000000,0.001050
1262,0.001400,0.000846,0.000000,0.002868,0.000454,0.000288,0.000689,0.000000,0.000692,0.000000,...,0.000166,0.000124,0.004739,0.000084,0.0,0.00,0.0,0.008876,0.000000,0.000000
1087,0.000911,0.000430,0.000000,0.001306,0.000907,0.000404,0.000689,0.000086,0.000207,0.000843,...,0.000099,0.001087,0.000872,0.000528,0.0,0.00,0.0,0.000000,0.000000,0.000000
707,0.004258,0.005387,0.000000,0.007405,0.009361,0.004541,0.011871,0.000000,0.001986,0.005367,...,0.002385,0.008346,0.004002,0.000021,0.0,0.00,0.0,0.000000,0.000096,0.000068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,0.015576,0.006375,0.000000,0.009026,0.006426,0.005162,0.006992,0.000000,0.005347,0.004903,...,0.005002,0.007237,0.014976,0.009901,0.0,0.25,0.0,0.020710,0.006889,0.001897
2234,0.000345,0.000437,0.000000,0.001140,0.000606,0.000717,0.000759,0.000000,0.000386,0.000843,...,0.000133,0.000708,0.000802,0.000042,0.0,0.00,0.0,0.000000,0.000000,0.000000
265,0.006023,0.008104,0.000000,0.027972,0.013759,0.006495,0.012793,0.000000,0.001519,0.001068,...,0.000000,0.015415,0.006185,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000
492,0.002532,0.001770,0.000000,0.002344,0.001413,0.000673,0.002194,0.000000,0.000917,0.000000,...,0.001027,0.000788,0.002228,0.001457,0.0,0.00,0.0,0.005917,0.001060,0.000169


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, full_shuffled_df['label'], stratify=full_shuffled_df['label'], test_size=0.2, random_state=42)

In [23]:
def ACCURACY(true, pred):   
    score = np.mean(true==pred)
    return score

In [24]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [25]:
data = full_shuffled_df.drop(['ID', 'esi','rtn', 'edx', 'eax', 'ebx', 'ecx', 'eip'], axis=1)
data.head()

,jmp,mov,retf,push,pop,xor,retn,nop,sub,inc,...,jz,lea,movzx,edi,ebp,esp,.dll,std::,:dword,label
2273,0.001400,0.000846,0.0,0.002868,0.000454,0.000288,0.000689,0.000000,0.000692,0.000000,...,0.000124,0.004739,0.000084,0.0,0.0,0.0,0.008876,0.000000,0.000000,1
2774,0.000374,0.000430,0.0,0.001238,0.000523,0.000743,0.000759,0.000129,0.000440,0.000885,...,0.000722,0.000862,0.000063,0.0,0.0,0.0,0.000000,0.000000,0.001050,1
1262,0.001400,0.000846,0.0,0.002868,0.000454,0.000288,0.000689,0.000000,0.000692,0.000000,...,0.000124,0.004739,0.000084,0.0,0.0,0.0,0.008876,0.000000,0.000000,1
1087,0.000911,0.000430,0.0,0.001306,0.000907,0.000404,0.000689,0.000086,0.000207,0.000843,...,0.001087,0.000872,0.000528,0.0,0.0,0.0,0.000000,0.000000,0.000000,1
707,0.004258,0.005387,0.0,0.007405,0.009361,0.004541,0.011871,0.000000,0.001986,0.005367,...,0.008346,0.004002,0.000021,0.0,0.0,0.0,0.000000,0.000096,0.000068,0


In [26]:
def kfold(model, train, scale = False):
    cv_accuracy = []
    cv = StratifiedKFold(n_splits=5)
    
    n_iter = 0
    
    for t, v in cv.split(train, train['label']):
        
        train_cv = train.iloc[t] # 훈련용
        val_cv = train.iloc[v] # 검증용 분리

        train_X = train_cv.drop('label', axis=1)
        train_y = train_cv['label']

        val_X = val_cv.drop('label', axis=1)
        val_y = val_cv['label']
            
        model.fit(train_X, train_y)
        score = ACCURACY(val_y, model.predict(val_X))
        
        cv_accuracy.append(score)
        n_iter += 1
    return np.mean(cv_accuracy)

In [27]:
# for LGBM error
import re
data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [35]:
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [36]:
models = [
    KNeighborsClassifier(),
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(max_depth=18, min_samples_leaf=2, min_samples_split=2, n_estimators=40, random_state=0, n_jobs=-1),
    GradientBoostingClassifier(),
    XGBClassifier(eval_metric = 'logloss', 
                              max_depth = 5, 
                              min_child_weight = 3, 
                               gamma = 3, 
                               colsample_bytree = 0.5, 
                               n_estimators=700),
    LGBMClassifier(n_estimators=600, max_bin=400, num_leaves=24),
    CatBoostClassifier(silent=True, depth=6, l2_leaf_reg=7, learning_rate=0.1, n_estimators=500),
    ExtraTreesClassifier(max_depth=25, n_estimators=320)
]

print('각 모델마다의 Kfold')
for model in models:
  print(f'{type(model).__name__} score: {kfold(model, data)}')

각 모델마다의 Kfold
KNeighborsClassifier score: 0.8686537018870102
LogisticRegression score: 0.6041497553875649
DecisionTreeClassifier score: 0.8711562506011118
RandomForestClassifier score: 0.9130276158783286
GradientBoostingClassifier score: 0.8979956527593437
XGBClassifier score: 0.9137451029424023
LGBMClassifier score: 0.9205455209956336
CatBoostClassifier score: 0.9237674803316214
ExtraTreesClassifier score: 0.9191163175409237


### Model Tuning

성능이 가장 잘 나온 상위 5개만 사용, 0.92로 많이 차이 나지 않는 경우 많이 쓰이는 모델로 사용  
Randomforest, LGBMClassifier, XGBClassifier, CatBoost, ExtraTrees

#### RandomForest

In [37]:
params = { 'n_estimators' : [10, 20, 30, 40, 50, 60, 70, 80],
           'max_depth' : [14, 16, 18, 20, 22],
           'min_samples_leaf' : [2, 3, 4],
           'min_samples_split' : [1, 2, 3, 4]
            }

In [38]:
train = full_shuffled_df.drop(['ID','esi','rtn', 'edx', 'eax', 'ebx', 'ecx', 'eip', 'label'], axis=1)
target = full_shuffled_df['label']

model_RFC = RandomForestClassifier()
grid_cv_RFC = GridSearchCV(model_RFC, param_grid = params, cv=5, n_jobs = -1)
grid_cv_RFC.fit(train, target)
print('최적 하이퍼 파라미터: ', grid_cv_RFC.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_RFC.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 80}
최고 예측 정확도: 0.9223


#### XGBClassifier

In [39]:
import warnings
warnings.filterwarnings(action="ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
model_XGB = XGBClassifier(eval_metric='logloss', silent = True)

param_grid={'booster' :['gbtree'],
                 'silent':[True],
                 'max_depth':[5,6,8],
                 'min_child_weight':[1,3,5],
                 'gamma':[0,1,2,3],
                 'nthread':[4],
                 'colsample_bytree':[0.5,0.8],
                 'colsample_bylevel':[0.9],
                 'n_estimators':[50],
                 'objective':['binary:logistic'],
                 'random_state':[2]}

grid_cv_XGB=GridSearchCV(model_XGB, param_grid=param_grid, cv=5 , n_jobs=-1)
grid_cv_XGB.fit(train, target)
print('최적 하이퍼 파라미터: ', grid_cv_XGB.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_XGB.best_score_))

최적 하이퍼 파라미터:  {'booster': 'gbtree', 'colsample_bylevel': 0.9, 'colsample_bytree': 0.8, 'gamma': 1, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 50, 'nthread': 4, 'objective': 'binary:logistic', 'random_state': 2, 'silent': True}
최고 예측 정확도: 0.9231

#### LGBM

In [55]:
model_LGBM = LGBMClassifier()

gridParams = {
    'learning_rate': [0.0001, 0.005, 0.01],
    'n_estimators': [2, 4, 6, 8],
    'num_leaves': [2, 3, 4, 5], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'random_state': [42],
    'subsample':[0.7, 0.75],
    'reg_alpha':[0.8, 1, 1.2],
    'reg_lambda':[0.8, 1,1.2],
    }

grid_cv_LGBM = GridSearchCV(model_LGBM, param_grid=gridParams , n_jobs=-1)
grid_cv_LGBM.fit(data, target)
print('최적 하이퍼 파라미터: ', grid_cv_LGBM.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_LGBM.best_score_))

최적 하이퍼 파라미터:  {'boosting_type': 'gbdt', 'learning_rate': 0.0001, 'max_bin': 255, 'n_estimators': 6, 'num_leaves': 2, 'objective': 'binary', 'random_state': 42, 'reg_alpha': 0.8, 'reg_lambda': 0.8, 'subsample': 0.7}
최고 예측 정확도: 1.0000


#### CatBoostClassifier

In [56]:
model_CAT = CatBoostClassifier(silent = True)

grid = {'learning_rate': [0.001, 0.01, 0.1],
        'depth': [2, 4, 6],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_cv_CAT = GridSearchCV(model_CAT, param_grid=grid, cv=3 , n_jobs=-1)
grid_cv_CAT.fit(data, target)
print('최적 하이퍼 파라미터: ', grid_cv_CAT.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_CAT.best_score_))

최적 하이퍼 파라미터:  {'depth': 2, 'l2_leaf_reg': 1, 'learning_rate': 0.001}
최고 예측 정확도: 1.0000


### ExtraClassifier

In [65]:
model_Extra = ExtraTreesClassifier()

grid_params = {
  'max_leaf_nodes':[2, 3, 4, 5, 6],
  'min_samples_split': [2, 3, 4],
  'random_state':[42],
  'n_estimators': [320, 640],
  'max_depth' : [10, 20, 30]
}

grid_cv_extra = GridSearchCV(model_Extra, param_grid=grid_params , n_jobs=-1)
grid_cv_extra.fit(data, target)
print('최적 하이퍼 파라미터: ', grid_cv_extra.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_extra.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 10, 'max_leaf_nodes': 2, 'min_samples_split': 2, 'n_estimators': 320, 'random_state': 42}
최고 예측 정확도: 0.9996


### Soft Voting

In [57]:
INCLUDE_LABEL = [  'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword', 'label']

X_train, X_test, y_train, y_test = train_test_split(data,full_shuffled_df['label'], stratify=full_shuffled_df['label'], test_size=0.2, random_state=42)

In [74]:
train = X_train.drop('label', axis=1)
target = X_train['label']
# for LGBM error
data = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
best_model_RF = RandomForestClassifier(max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=80, random_state=42)
best_model_XGB = XGBClassifier(eval_metric = 'logloss', 
                              booster = 'gbtree',
                              colsample_bylevel=0.9,
                              colsample_bytree=0.8,
                              silent = True, 
                              max_depth = 6, 
                              min_child_weight = 3, 
                               gamma = 1, 
                               n_estimators=50,
                               nthread=4,
                               objective='binary:logistic',
                               random_state=42,
                               learning_rate=0.0001,
                               num_leaves=2,
                               )
best_model_LGBM = LGBMClassifier(boosting_type='gbdt',learning_rate=0.0001 ,n_estimators=6, max_bin=255, num_leaves=2,objective='binary',random_state=42, reg_alpha=0.8, reg_lambda=0.8, subsample=0.7)
best_model_CAT = CatBoostClassifier(silent=True, depth=2, l2_leaf_reg=1, learning_rate=0.0001)
best_model_EXTRA = ExtraTreesClassifier(max_depth=10, max_leaf_nodes=2, min_samples_split=2, n_estimators=320, random_state=42)

In [68]:
from sklearn.ensemble import VotingClassifier
softVoting_model = VotingClassifier(estimators=[('XGB', best_model_XGB), ('LGBM', best_model_LGBM), ('CAT', best_model_CAT), ('EXTRA', best_model_EXTRA), ('RF', best_model_RF)], voting='soft')
softVoting_model.fit(data, target)

[02:45:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1644955194972/work/src/learner.cc:576: 
Parameters: { "num_leaves", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




VotingClassifier(estimators=[('XGB',
                              XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=0.9,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8,
                                            enable_categorical=False,
                                            eval_metric='logloss', gamma=1,
                                            gpu_id=None, importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.0001,
                                            max_delta_step=None, max_depth=6,
                                            min_child_weight=3, missing=nan,
                                            m...
                                             objective='binary',
                                             rando

In [69]:
X_test = X_test.drop('label', axis=1)

In [75]:
from sklearn.metrics import classification_report, confusion_matrix
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

soft_pred = softVoting_model.predict(X_test)
print(classification_report(y_test, soft_pred))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93       280
           1       0.94      0.91      0.93       279

    accuracy                           0.93       559
   macro avg       0.93      0.93      0.93       559
weighted avg       0.93      0.93      0.93       559

